# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# Data

In [5]:
food = pd.read_csv('../scrapdata/FoodBalanceSheets_E_All_Data.csv', encoding = 'latin1')

In [6]:
food.head()

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2009,Y2009F,Y2010,Y2010F,Y2011,Y2011F,Y2012,Y2012F,Y2013,Y2013F
0,2,Afghanistan,2501,Population,511,Total Population - Both sexes,1000 persons,8954.0,NaN,9142.0,...,27708.0,NaN,28398.0,NaN,29105.0,NaN,29825.0,NaN,30552.0,NaN
1,2,Afghanistan,2511,Wheat and products,5511,Production,1000 tonnes,2279.0,S,2279.0,...,5064.0,S,4532.0,S,3388.0,S,5050.0,S,5169.0,S
2,2,Afghanistan,2511,Wheat and products,5611,Import Quantity,1000 tonnes,32.0,S,7.0,...,1861.0,S,1264.0,S,1759.0,S,1334.0,S,1173.0,S
3,2,Afghanistan,2511,Wheat and products,5072,Stock Variation,1000 tonnes,0.0,S,0.0,...,-1300.0,S,-200.0,S,400.0,S,-500.0,S,-350.0,S
4,2,Afghanistan,2511,Wheat and products,5301,Domestic supply quantity,1000 tonnes,2311.0,S,2286.0,...,5625.0,S,5596.0,S,5547.0,S,5884.0,S,5992.0,S
